Get Neighbourhood data and coordinates

In [1]:
import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
Neighbourhood = []
#read the table into an array
for row in My_table.find_all('tr'):
    col = row.findAll('td')
    if len(col) > 0:
        postcode = col[0].string
        county = col[1].string
        borough = col[2].string
        Neighbourhood.append([t.text.strip() for t in row.find_all('td')])
        
df = pd.DataFrame([])
#populate the dataframe from array
df = pd.DataFrame(Neighbourhood, columns=['PostalCode', 'Borough', 'Neighbourhood'])
 # Get names of indexes for which column 'Borough is Not assigned'
indexNames = df[ df['Borough'] == 'Not assigned' ].index
    
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head() 
df['Neighbourhood'] = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].transform(lambda x: ', '.join(set(x.dropna())))
df.drop_duplicates()
df.loc[df['Neighbourhood'] =='Not assigned','Neighbourhood']=df['Borough']
#Get coordinates
# Load the Pandas libraries with alias 'pd' 
# Read data from file 
data = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_coordinates.csv") 
# change the column name of the first column
data.columns = ['PostalCode','Latitude','Logitude']
#Merge the data of two dataframes
df_final = pd.merge(df, data, on='PostalCode')

import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
Get latitude longitude of Toronto

SyntaxError: invalid syntax (<ipython-input-3-020a199aee7e>, line 1)

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of toronto with the latitude, logitude obtained

In [5]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#Slice the dataframe to get only toronto borogh
df_toronto = df_final[df_final['Borough'].str.contains (u'Toronto')].reset_index(drop=True)
print(df_toronto.head())
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Logitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

  PostalCode           Borough              Neighbourhood   Latitude  \
0        M5A  Downtown Toronto  Regent Park, Harbourfront  43.654260   
1        M5A  Downtown Toronto  Regent Park, Harbourfront  43.654260   
2        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
3        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
4        M5C  Downtown Toronto             St. James Town  43.651494   

    Logitude  
0 -79.360636  
1 -79.360636  
2 -79.378937  
3 -79.378937  
4 -79.375418  


Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'ROZBATJPUJS2EJ55CETLI23AFTD3JLESDQNMD4SA1TXVX51C' # your Foursquare ID
CLIENT_SECRET = 'L4U0MC1QO0BLQREMSCGT0MPWPESGMPY320VCFMTS5HALUGMH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


get latitude and longotude of regent park

In [7]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Logitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


get the top 10 venues that are in Regent park within a radius of 100 meters.

In [8]:
radius = 500 # define radius
LIMIT  =10
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6b1c792c20174761a7bd2d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Get the data into a dataframe

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
8,Corktown Common,Park,43.655618,-79.356211
9,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


Create and call a function to get venues for each neighbourhood

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        nearby_venues = json_normalize(venues) # flatten JSON
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get the venues for each neighbourhood

In [19]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Logitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Regent Park, Harbourfront
Regent Park, Harbourfront
Ryerson, Garden District
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, King, Adelaide
Richmond, King, Adelaide
Richmond, King, Adelaide
Dovercourt Village, Dufferin
Dovercourt Village, Dufferin
Toronto Islands, Union Station, Harbourfront East
Toronto Islands, Union Station, Harbourfront East
Toronto Islands, Union Station, Harbourfront East
Little Portugal, Trinity
Little Portugal, Trinity
Riverdale, The Danforth West
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Toronto Dominion Centre, Design Exchange
Exhibition Place, Brockton, Parkdale Village
Exhibition Place, Brockton, Parkdale Village
Exhibition Place, Brockton, Parkdale Village
The Beaches West, India Bazaar
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
Forest 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.head())
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

   Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   American Restaurant  Arts & Crafts Store  Asian Restaurant  Auto Workshop  \
0                    0                    0                 0              0   
1                    0                    0                 0              0   
2                    0                    0                 0              0   
3                    0                    0                 0              0   
4                    0                    0                 0              0   

   BBQ Joint  Baby Store  Bakery  Bar  Beer Bar  Booksto

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Museum,Music Venue,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0
1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
2,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.1,0.1,0.1,0.10,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.10,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.3,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0

cluster the neighbourhoods

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Function to sort the venues desc

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

get the sorted venues in a dataframe

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,French Restaurant,Steakhouse,Museum,Concert Hall,Liquor Store,Cocktail Bar,Farmers Market,Beer Bar,Park,Vegetarian / Vegan Restaurant
1,Business Reply Mail Processing Centre 969 Eastern,Garden Center,Burrito Place,Brewery,Farmers Market,Fast Food Restaurant,Restaurant,Auto Workshop,Pizza Place,Comic Shop,Skate Park
2,"Cabbagetown, St. James Town",Café,Italian Restaurant,Jewelry Store,Indian Restaurant,Diner,Bakery,Restaurant,Japanese Restaurant,Butcher,Creperie
3,Central Bay Street,Coffee Shop,Gastropub,Italian Restaurant,Park,Sushi Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Spa,Comic Shop,Concert Hall
4,Christie,Café,Grocery Store,Coffee Shop,Restaurant,Baby Store,Italian Restaurant,Diner,Dog Run,College Gym,Comic Shop


display the dataframe that includes cluster and top 10 venues

In [28]:
toronto_merged.describe

<bound method NDFrame.describe of    PostalCode           Borough  \
0         M5A  Downtown Toronto   
1         M5A  Downtown Toronto   
2         M5B  Downtown Toronto   
3         M5B  Downtown Toronto   
4         M5C  Downtown Toronto   
5         M4E      East Toronto   
6         M5E  Downtown Toronto   
7         M5G  Downtown Toronto   
8         M6G  Downtown Toronto   
9         M5H  Downtown Toronto   
10        M5H  Downtown Toronto   
11        M5H  Downtown Toronto   
12        M6H      West Toronto   
13        M6H      West Toronto   
14        M5J  Downtown Toronto   
15        M5J  Downtown Toronto   
16        M5J  Downtown Toronto   
17        M6J      West Toronto   
18        M6J      West Toronto   
19        M4K      East Toronto   
20        M4K      East Toronto   
21        M5K  Downtown Toronto   
22        M5K  Downtown Toronto   
23        M6K      West Toronto   
24        M6K      West Toronto   
25        M6K      West Toronto   
26        M4L      Ea

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Logitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Gym / Fitness Center,Breakfast Spot,Bakery,Restaurant,Pub,Spa,Coffee Shop,Park,Concert Hall,Cosmetics Shop
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Gym / Fitness Center,Breakfast Spot,Bakery,Restaurant,Pub,Spa,Coffee Shop,Park,Concert Hall,Cosmetics Shop
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Café,Clothing Store,Ramen Restaurant,Theater,Pizza Place,Tea Room,Burrito Place,Plaza,Comic Shop,Creperie
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Café,Clothing Store,Ramen Restaurant,Theater,Pizza Place,Tea Room,Burrito Place,Plaza,Comic Shop,Creperie
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Gastropub,Gym,Middle Eastern Restaurant,Restaurant,Creperie,Italian Restaurant,Japanese Restaurant,Food Truck,Flea Market


create map

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Logitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the clusters

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,0,Park,Trail,Sushi Restaurant,Jewelry Store,Dog Run,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall
35,Central Toronto,0,Park,Trail,Sushi Restaurant,Jewelry Store,Dog Run,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall
49,Central Toronto,0,Park,Playground,Tennis Court,Summer Camp,Diner,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop
50,Central Toronto,0,Park,Playground,Tennis Court,Summer Camp,Diner,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop
66,Downtown Toronto,0,Park,Trail,Playground,Building,Diner,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Gym / Fitness Center,Breakfast Spot,Bakery,Restaurant,Pub,Spa,Coffee Shop,Park,Concert Hall,Cosmetics Shop
1,Downtown Toronto,1,Gym / Fitness Center,Breakfast Spot,Bakery,Restaurant,Pub,Spa,Coffee Shop,Park,Concert Hall,Cosmetics Shop
2,Downtown Toronto,1,Café,Clothing Store,Ramen Restaurant,Theater,Pizza Place,Tea Room,Burrito Place,Plaza,Comic Shop,Creperie
3,Downtown Toronto,1,Café,Clothing Store,Ramen Restaurant,Theater,Pizza Place,Tea Room,Burrito Place,Plaza,Comic Shop,Creperie
4,Downtown Toronto,1,Coffee Shop,Gastropub,Gym,Middle Eastern Restaurant,Restaurant,Creperie,Italian Restaurant,Japanese Restaurant,Food Truck,Flea Market
7,Downtown Toronto,1,Coffee Shop,Gastropub,Italian Restaurant,Park,Sushi Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Spa,Comic Shop,Concert Hall
8,Downtown Toronto,1,Café,Grocery Store,Coffee Shop,Restaurant,Baby Store,Italian Restaurant,Diner,Dog Run,College Gym,Comic Shop
14,Downtown Toronto,1,Park,Supermarket,Sporting Goods Shop,Salad Place,Bubble Tea Shop,Lake,Dessert Shop,Café,Performing Arts Venue,Comic Shop
15,Downtown Toronto,1,Park,Supermarket,Sporting Goods Shop,Salad Place,Bubble Tea Shop,Lake,Dessert Shop,Café,Performing Arts Venue,Comic Shop
16,Downtown Toronto,1,Park,Supermarket,Sporting Goods Shop,Salad Place,Bubble Tea Shop,Lake,Dessert Shop,Café,Performing Arts Venue,Comic Shop


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,2,Garden,Wine Bar,Eastern European Restaurant,Coffee Shop,College Gym,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,3,Park,Bus Line,Swim School,Lawyer,Wine Bar,Diner,Coffee Shop,College Gym,Comic Shop,Concert Hall


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,East Toronto,4,Trail,Pub,Health Food Store,Wine Bar,Dog Run,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall
6,Downtown Toronto,4,French Restaurant,Steakhouse,Museum,Concert Hall,Liquor Store,Cocktail Bar,Farmers Market,Beer Bar,Park,Vegetarian / Vegan Restaurant
9,Downtown Toronto,4,Steakhouse,Concert Hall,Vegetarian / Vegan Restaurant,Food Court,Pizza Place,Speakeasy,Asian Restaurant,Hotel,Plaza,Dance Studio
10,Downtown Toronto,4,Steakhouse,Concert Hall,Vegetarian / Vegan Restaurant,Food Court,Pizza Place,Speakeasy,Asian Restaurant,Hotel,Plaza,Dance Studio
11,Downtown Toronto,4,Steakhouse,Concert Hall,Vegetarian / Vegan Restaurant,Food Court,Pizza Place,Speakeasy,Asian Restaurant,Hotel,Plaza,Dance Studio
12,West Toronto,4,Bakery,Pharmacy,Café,Bar,Supermarket,Gym / Fitness Center,Brewery,Middle Eastern Restaurant,Music Venue,Concert Hall
13,West Toronto,4,Bakery,Pharmacy,Café,Bar,Supermarket,Gym / Fitness Center,Brewery,Middle Eastern Restaurant,Music Venue,Concert Hall
17,West Toronto,4,Wine Bar,Asian Restaurant,Pizza Place,Cuban Restaurant,Korean Restaurant,Brewery,Ice Cream Shop,Bar,Cocktail Bar,Greek Restaurant
18,West Toronto,4,Wine Bar,Asian Restaurant,Pizza Place,Cuban Restaurant,Korean Restaurant,Brewery,Ice Cream Shop,Bar,Cocktail Bar,Greek Restaurant
19,East Toronto,4,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Pub,Italian Restaurant,Brewery,Dog Run,College Gym,Comic Shop
